In [1]:
import os
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [2]:
load_dotenv()  # Load environment variables from .env file
HF_TOKEN = os.getenv("HF_TOKEN")
huggingface_repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

In [3]:
#model loading
llm =HuggingFaceEndpoint(repo_id=huggingface_repo_id,temperature=0.3,model_kwargs={"token":HF_TOKEN,"max_length":512})

D:\anaconda\envs\project_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
#custom prompt

custom_prompt_template = """
Use the information provided in the context to answer the user's question about skin diseases.
If you don't know the answer, provide a general response based on common knowledge about skin diseases.
Don't provide anything outside of the given context.

Context: {context}
Question: {question}

Provide a direct answer, clear and concise, without unnecessary elaboration.
"""
prompt = PromptTemplate(template=custom_prompt_template, input_variables=["context","question"])

In [5]:
DB_FAISS_PATH = 'vectorstore/db_faiss'
embedding_model =HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
try:
    db = FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)
except Exception as e:
    print(f"Error loading FAISS index: {e}")

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={'k': 3}),
    chain_type_kwargs={'prompt': prompt},
    chain_type="stuff"  
)


In [12]:
query = input("enter your question:")
result = qa_chain.invoke({"query":query})
print(result['result'])

enter your question: what is skin



Skin is the largest organ that covers the entire surface of the body, made up of three main layers: the epidermis, dermis, and subcutaneous tissue. It is a mirror of the body, often reflecting one's health status.


In [18]:
query = input('asksomething')
result = qa_chain.invoke({"query": query})  # Use 'invoke' instead of 'run'
print("Generated Answer:", result["result"])


Generated Answer: 
Molluscum contagiosum is a viral infection, not a bacterial or fungal infection. It is caused by the pox virus.
